In [1]:
from requests import get
from bs4 import BeautifulSoup

In [2]:
#creating loop to create urls for 50 movies at a time

#initializing iterator variable (pages increment in form of 'start=1', then 'start=51', then 'start=101'......)
z = 1

#creating empty list for urls to be stored in
urls = []

#while the number of movies is below 4002 (arbitrary #)
while z < 4002:
    
    #generating url using format to insert the incrementing variable on each loop
    url = 'https://www.imdb.com/search/title/?title_type=feature&moviemeter=,4000&sort=user_rating,desc&start={}&ref_=adv_nxt'.format(z)
    #try and use this url at some point
    #https://www.imdb.com/search/title/?title_type=feature&release_date=1950-01-01,2020-06-30&user_rating=4.0,&start=1&ref_=adv_nxt
    
    #appending the url to the list of urls
    urls.append(url)
    
    #incrementing iterator variable so it can go to the next page (there are 50 movies listed per page)
    z += 50


In [ ]:
#for loop to get all the html objects of each array into a soup object

#creating empty soup item to store the generated html files in
html_soups = BeautifulSoup()

#iterating through the url pages
for url in urls:
    
    #creating response to use in BeautifulSoup function
    response = get(url)
    
    #using the BeautifulSoup function on each response, appending the resulting soup object into the empty 'html_soups'
    html_soups.append(BeautifulSoup(response.text, 'html.parser'))

In [4]:
#creating containers for each movie, seperating at 'lister-item mode-advanced' on html doc
m_containers = html_soups.find_all('div', class_='lister-item mode-advanced')

#printing to make sure the results are what we expect; if you go to the URL, there are only 2758 items on the list created, all are here
print(type(m_containers))
print(len(m_containers))

<class 'bs4.element.ResultSet'>
2758


In [51]:


for a in container.find_all('a', href=True):
        movie_link = ('imdb.com'+a['href'])
        stars.append(movie_link)

In [99]:
stars = []
j=0
for container in m_containers:
    
    #link = container.h3.find_all('a', href=True)
    
    #NEED TO FIGURE THIS OUT!!! WHAT IS THE PROBLEM WHY ARENT THE STARS ABLE TO BE ACCCESSED!!????!?!?!?
    #USE THE COMMAS!!: if there are 3 commas, there are four stars, 2 commas = 3 stars yadda yadda
    #while i < 5:
        
    #
    for a in container.h3.find_all('a', href=True):
        m_link = ('imdb.com'+a['href'])
        print(m_link)
    
    for i in range(len(container.find_all('p')[2].find_all('a'))):
        if container.find_all('p')[2].find_all('a') is not None and len(container.find_all('p')[2].find_all('a')) > 0:
            print(len(container.find_all('p')[2].find_all('a')))
            star = container.find_all('p')[2].find_all('a')[i].text
            print(star)
    #for a in container.find_all('p')[2]:
    #comma = container.find_all('p')[2].find_all('a')[1].text
        
    #star = container.find_all('p')[2].find_all('a')[1].text
    #print(star)
    #star2 = container.find_all('p')[2].find_all('a')[2].text
    #print(star2)
    j+=1
    print(j)
    #stars.append(star)
    #stars.append(star2)
    
    



imdb.com/title/tt0111161/
5
Frank Darabont
5
Tim Robbins
5
Morgan Freeman
5
Bob Gunton
5
William Sadler
1
imdb.com/title/tt0068646/
5
Francis Ford Coppola
5
Marlon Brando
5
Al Pacino
5
James Caan
5
Diane Keaton
2
imdb.com/title/tt0468569/
5
Christopher Nolan
5
Christian Bale
5
Heath Ledger
5
Aaron Eckhart
5
Michael Caine
3
imdb.com/title/tt0071562/
5
Francis Ford Coppola
5
Al Pacino
5
Robert De Niro
5
Robert Duvall
5
Diane Keaton
4
imdb.com/title/tt0167260/
5
Peter Jackson
5
Elijah Wood
5
Viggo Mortensen
5
Ian McKellen
5
Orlando Bloom
5
imdb.com/title/tt0110912/
5
Quentin Tarantino
5
John Travolta
5
Uma Thurman
5
Samuel L. Jackson
5
Bruce Willis
6
imdb.com/title/tt0108052/
5
Steven Spielberg
5
Liam Neeson
5
Ralph Fiennes
5
Ben Kingsley
5
Caroline Goodall
7
imdb.com/title/tt0050083/
5
Sidney Lumet
5
Henry Fonda
5
Lee J. Cobb
5
Martin Balsam
5
John Fiedler
8
imdb.com/title/tt1375666/
5
Christopher Nolan
5
Leonardo DiCaprio
5
Joseph Gordon-Levitt
5
Ellen Page
5
Ken Watanabe
9
imdb.com/tit

In [44]:
#creating empty arrays to store 
titles = []
years = []
imdb_ratings = []
summs = []
mpaa_rates = []


In [54]:
#creating empty list to store dictionaries of each movie info
movies = []

#for loop to parse through each movie in m_containers and get the info
for container in m_containers:
    
    #finding title of movie through its location in html doc
    title = container.h3.a.text
    
    #finding link for each movies imdb page
    #for loop to go through the container and find the correct link
    for a in container.h3.find_all('a', href=True):
        m_link = ('imdb.com'+a['href'])
    #appending title to empty title list
    #titles.append(title)

    
    #finding year of movie through its location in html doc
    year = container.h3.find('span', class_='lister-item-year').text
    
    #appending year to empty year list
    #years.append(year)

    
    #if function created to deal with any movies that dont have ratings
   
    #if there is a rating found in the html
    if container.strong is not None:
        
        #imdb rating stored in i_rate
        i_rate = float(container.strong.text)
        
    #if there is NO imdb rating:
    else:
        
        #set i_rate to 0
        i_rate = 0

    #append the value to list
    #imdb_ratings.append(i_rate)
        
    #finding mpaa rating (G/PG/R etc.) in html
    rating = container.p.span.text
    
    #appending mpaa rating to mpaa_rates list
    #mpaa_rates.append(rating)
    
    #finding director in html
    if container.find_all('p')[2].a is not None:
        dir = container.find_all('p')[2].a.text
    else:
        dir = 'NA'

    #finding summary of movie in html
    summary = container.find_all('p', class_='text-muted')[1].text.strip('\n   ')
    
    #appending summary to summs list
    #summs.append(summary)
    
   
    
    #appending the values for movie to dictionaries, creating new 'movie' class for each movie
    movies.append({'movie':{'Title':title,'Movie Link':m_link, 'Year':year, 'IMDB Rating':i_rate, "Summary": summary, "MPAA Rating":rating, "Director":dir}})

In [56]:
movies[-1:]

[{'movie': {'Title': 'Mortal Kombat',
   'Movie Link': 'imdb.com/title/tt0293429/',
   'Year': '(2021)',
   'IMDB Rating': 0,
   'Summary': 'Plot unknown. Feature film based on the popular video game.',
   'MPAA Rating': '\nAction, Adventure, Fantasy            ',
   'Director': 'Simon McQuoid'}}]

In [ ]:
#checking if all le
print(len(titles))
print(len(summs))
print(len(years))
print(len(imdb_ratings))
print(len(mpaa_rates))

To do:
1. Fix the stars section, how to add them to dict

2. find a way to get the documents transferred to mongodb

3. Begin process of creating functionality - research how to do the things we want to do

4. Optimize speed - could the m_containers loop be inside of the "for url in urls" loop? that may make the operation faster

In [ ]:
import pandas as pd
df = pd.DataFrame({'movie': titles,
'year': years,
'imdb': imdb_ratings,
'summary': summs,
'mpaa_rates': mpaa_rates
})

#removing movies with 0's as they arent out yet...
#df = df[df.imdb != 0]


In [ ]:
titles_years = [{"Title": t, "Year": s} for t, s in zip(titles, years)]
titles_years[:11]

In [ ]:
import json
print(json.dumps(movies))